In [80]:
import pandas as pd
import json
from pycarol import loginCarol, entityTemplateCarol


with open('../carol-ds-retail/config.json') as json_data:
    d = json.loads(json_data.read())
token_object = loginCarol.loginCarol(**d['mario'])
token_object.newToken()
print(token_object.access_token)

aedcce20ce1911e784630242ac110003


In [81]:
token_object.refresh_token

'aedcce21ce1911e784630242ac110003'

In [82]:
token_object.refreshToken()

In [94]:
json_query = """{"tenantId":"e7dbd874-ea9d-4092-afa4-d2f68acaaaf4","id":1,"createdAt":"2015-11-16T17:34:37-02:00",
"subtotal":410.1,"discountsAndFees":[],"interestFee":0,"payInFullDiscount":0,"interestRate":0,"quantityOfInstallments":1,
"shippingMethod":{"gatewayId":1,"id":6723,"name":"Frete Padrão","deliveryDeadLine":"5 dias","cost":1,"adjustedCost":1},
"total":411.1,"shippingAddress":{"id":1,"recipientName":"José da Silva","local":"José da Silva","phoneAreaCode":"41",
"phoneNumber":"3333-3333","phoneAreaCodeAlternative":null,
"streetNumber":"100"},
"package":{"id":null,"description":null,"value":0},"paymentMethods":[{"id":54,"name":"Faturado","installments":1,"value":411.1,"sequential":1,"paymentMethodGroup":{"id":25,"name":"Interno"},"paymentMethodCategory":{"id":13,"name":"Faturado"}}]}"""
json_sample = json.loads(json_query)
json_sample

{'createdAt': '2015-11-16T17:34:37-02:00',
 'discountsAndFees': [],
 'id': 1,
 'interestFee': 0,
 'interestRate': 0,
 'package': {'description': None, 'id': None, 'value': 0},
 'payInFullDiscount': 0,
 'paymentMethods': [{'id': 54,
   'installments': 1,
   'name': 'Faturado',
   'paymentMethodCategory': {'id': 13, 'name': 'Faturado'},
   'paymentMethodGroup': {'id': 25, 'name': 'Interno'},
   'sequential': 1,
   'value': 411.1}],
 'quantityOfInstallments': 1,
 'shippingAddress': {'id': 1,
  'local': 'José da Silva',
  'phoneAreaCode': '41',
  'phoneAreaCodeAlternative': None,
  'phoneNumber': '3333-3333',
  'recipientName': 'José da Silva',
  'streetNumber': '100'},
 'shippingMethod': {'adjustedCost': 1,
  'cost': 1,
  'deliveryDeadLine': '5 dias',
  'gatewayId': 1,
  'id': 6723,
  'name': 'Frete Padrão'},
 'subtotal': 410.1,
 'tenantId': 'e7dbd874-ea9d-4092-afa4-d2f68acaaaf4',
 'total': 411.1}

In [37]:
ent = entityTemplateCarol.createTemplate(token_object)

In [38]:
ent.addFields(json_sample)

In [39]:
ent.possible_types

['boolean', 'date', 'long', 'double', 'nested', 'string', 'object', 'geopoint']

In [52]:
class entIntType(object):
    ent_type = 'long'


class entDoubleType(object):
    ent_type = 'double'


class entStringType(object):
    ent_type = "string"


class entNullType(object):
    ent_type = "string"


class entBooleanType(object):
    ent_type = "boolean"


class entArrayType(object):
    ent_type = "nested"


class entObjectType(object):
    ent_type = "object"



class entType(object):
    @classmethod
    def get_ent_type_for(cls, t):
        """docstring for get_schema_type_for"""
        SCHEMA_TYPES = {
            type(None): entNullType,
            str: entStringType,
            int: entIntType,
            float: entDoubleType,
            bool: entBooleanType,
            list: entArrayType,
            dict: entObjectType,
        }

        schema_type = SCHEMA_TYPES.get(t)

        if not schema_type:
            raise JsonEntTypeNotFound("There is no schema type for  %s.\n Try:\n %s" % (
            str(t), ",\n".join(["\t%s" % str(k) for k in SCHEMA_TYPES.keys()])))
        return schema_type


class JsonEntTypeNotFound(Exception):
    pass

In [41]:
SCHEMA_TYPES = {
    type(None): NullType,
    str: StringType,
    int: IntType,
    float: DoubleType,
    bool: BooleanType,
    list: ArrayType,
    dict: ObjectType,
}

NameError: name 'NullType' is not defined

In [64]:
ent_ =  ent.all_possible_fields.get('id',[]).copy()
ent_.pop('mdmCreated') 
ent_.pop('mdmLastUpdated') 
ent_.pop('mdmTenantId')
ent_

{'mdmAnalyzer': 'STANDARD',
 'mdmDescription': {},
 'mdmEntityType': 'mdmField',
 'mdmFieldType': 'PRIMITIVE',
 'mdmFields': [],
 'mdmFieldsFull': {},
 'mdmForeignKeyField': False,
 'mdmId': '1da76a80a85f11e7a2900242ac110003',
 'mdmIndex': 'ANALYZED',
 'mdmLabel': {'en-US': 'id'},
 'mdmMappingDataType': 'LONG',
 'mdmName': 'id',
 'mdmTags': []}

In [97]:
def create_field(mdmName,value,entity_type,parentId = ''): 
    payload = {"mdmName":mdmName,"mdmMappingDataType":entity_type.ent_type,
           "mdmLabel":{"en-US": mdmName},"mdmDescription":{"en-US": mdmName}}
    
    if entity_type.ent_type == 'nested':
        payload['mdmFieldType'] = 'NESTED'
        print('criando NESTED')
        parentId = 1234
        _, parentId = create_field()
    else:
        payload['mdmFieldType'] = 'PRIMITIVE'   
        print('criando PRIMITIVE')

    return payload, parentId

for prop, value in json_sample.items():
    entity_type = entType.get_ent_type_for(type(value))
    if prop in ent.all_possible_fields.keys():
        ent_ =  ent.all_possible_fields.get(prop,[]).copy()
        ent_.pop('mdmCreated') 
        ent_.pop('mdmLastUpdated') 
        ent_.pop('mdmTenantId') 
        if (ent_['mdmMappingDataType'].lower() == entity_type.ent_type) :
            print('SEND FIELD TO DM CREATED')
        else:
            raise('problem')
    else:
        to_create = create_field(prop,value, entity_type)
        print(to_create)

criando PRIMITIVE
({'mdmName': 'tenantId', 'mdmMappingDataType': 'string', 'mdmLabel': {'en-US': 'tenantId'}, 'mdmDescription': {'en-US': 'tenantId'}, 'mdmFieldType': 'PRIMITIVE'}, '')
SEND FIELD TO DM CREATED
criando PRIMITIVE
({'mdmName': 'createdAt', 'mdmMappingDataType': 'string', 'mdmLabel': {'en-US': 'createdAt'}, 'mdmDescription': {'en-US': 'createdAt'}, 'mdmFieldType': 'PRIMITIVE'}, '')
criando PRIMITIVE
({'mdmName': 'subtotal', 'mdmMappingDataType': 'double', 'mdmLabel': {'en-US': 'subtotal'}, 'mdmDescription': {'en-US': 'subtotal'}, 'mdmFieldType': 'PRIMITIVE'}, '')
criando NESTED


TypeError: create_field() missing 3 required positional arguments: 'mdmName', 'value', and 'entity_type'

In [101]:
ent.all_possible_fields['mdmemail']

{'mdmAnalyzer': 'STANDARD',
 'mdmCreated': '2017-10-21T04:52:46Z',
 'mdmDescription': {'en-US': 'Email', 'pt-BR': 'Email'},
 'mdmEntityType': 'mdmField',
 'mdmFieldType': 'NESTED',
 'mdmFields': [{'mdmCustomDescription': {'en-US': 'Email Address',
    'pt-BR': 'Endereço Email'},
   'mdmCustomLabel': {'en-US': 'Email Address', 'pt-BR': 'Endereço Email'},
   'mdmEnableAddressCleansing': False,
   'mdmFieldSurvivorshipRuleIds': [],
   'mdmFieldSurvivorshipRules': [],
   'mdmFieldType': 'PRIMITIVE',
   'mdmFields': [],
   'mdmFlagRuleIds': [],
   'mdmFlagRules': [],
   'mdmForeignKeyField': False,
   'mdmHasMapping': False,
   'mdmId': 'e9aa7ab0b68c11e794e30242ac110003',
   'mdmInternalField': False,
   'mdmMappingDataType': 'STRING',
   'mdmName': 'mdmemailaddress',
   'mdmRequiredBy': '',
   'mdmSkipRuleIds': [],
   'mdmSkipRules': [],
   'mdmTags': []},
  {'mdmCustomDescription': {'en-US': 'Email Type', 'pt-BR': 'Tipo do Email'},
   'mdmCustomLabel': {'en-US': 'Type', 'pt-BR': 'Tipo'},


In [65]:
a = {'mdmAnalyzer':'STANDARD','mdmDescription':{},'mdmEntityType':'mdmField','mdmFieldType':'PRIMITIVE',
                 'mdmFields':[],'mdmFieldsFull':{},'mdmForeignKeyField':False,'mdmId':'1da76a80a85f11e7a2900242ac110003',
                 'mdmIndex':'ANALYZED','mdmLabel':{'en-US':'id'},'mdmMappingDataType':'LONG','mdmName':'id','mdmTags':[]}

In [88]:
json_sample.keys()

dict_keys(['tenantId', 'id', 'createdAt', 'updatedAt', 'deliveredAt', 'deliveryEstimatedDate', 'status', 'cancellationReason', 'statusMsgStore', 'statusMsgCustomer', 'subtotal', 'discountsAndFees', 'interestFee', 'payInFullDiscount', 'interestRate', 'quantityOfInstallments', 'appliedPromotions', 'shippingMethod', 'total', 'totalOriginal', 'fullfillmentType', 'storePickup', 'shippingAddress', 'isGuestPurchase', 'customerId', 'billing', 'ip', 'referrerUrl', 'storeURL', 'source', 'customer', 'orderItems', 'shipments', 'multiShipments', 'transactionType', 'subscriptonToken', 'cartId', 'checkStockId', 'paymentTransactions', 'pricingTable', 'sourceChannel', 'invoices', 'templateId', 'wishlist', 'marketplace', 'appClientId', 'sessionId', 'paymentMethods'])

In [72]:
json_sample.pop('')

In [73]:
a

{'mdmAnalyzer': 'STANDARD',
 'mdmDescription': {},
 'mdmEntityType': 'mdmField',
 'mdmFieldType': 'PRIMITIVE',
 'mdmFields': [],
 'mdmFieldsFull': {},
 'mdmForeignKeyField': False,
 'mdmId': 1234,
 'mdmIndex': 'ANALYZED',
 'mdmLabel': {'en-US': 'id'},
 'mdmMappingDataType': 'LONG',
 'mdmName': 'id',
 'mdmTags': [],
 'teste': 'sdasd'}